<a href="https://colab.research.google.com/github/elidonda2-web/Alisia-7B-it-/blob/main/Chat_with_Alisia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Au cas où si il y pas un dossier nommé drive dans **Afficher l'explorateur des fichiers** clique sur c'est code:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q gradio
!pip install -q llama-cpp-python

##Remplace la place de **votre idee** par le lien que vous avez copié dans Drive.
**NB:** c'est utile seulement dans kaggle mais dans Drive c'est pas nécessaire voici le code:

In [ ]:
!pip install gdown --quit
!gdown Votre_id -O /kaggle/working/Alisia.Q4_K_M.gguf

##Dans Drive:
**MODEL_PATH="/content/drive/MyDrive/Q
Alisia.Q4_K_M.gguf**
——————————————————————
##Mais dans kaggle:
**MODEL_PATH="/kaggle/working/Alisia.Q4_K_M.gguf**

In [ ]:
# **********************************************************************
# Alisis - Chat UI complete (GGUF + Streaming) - VERSION CORRIGÉE
# **********************************************************************
from llama_cpp import Llama
import gradio as gr
import json
import os
import datetime

# **********************************************************************
# Paramètres du modèle
# **********************************************************************

MODEL_PATH = "/content/drive/MyDrive/Alisia.Q4_K_M.gguf"  # ✅ tu vas remplacer quand du sera dans kaggle par /kaggle/working/Alisia.Q4_K_M.gguf

# Vérification du fichier modèle
if not os.path.exists(MODEL_PATH):
    print(f"❌ Fichier modèle non trouvé: {MODEL_PATH}")
    print("📁 Fichiers dans le répertoire courant:")
    for file in os.listdir("."):
        if file.endswith(".gguf"):
            print(f"  - {file}")
    # Si le modèle n'existe pas, on crée un simulateur pour tester l'interface
    llm = None
    print("⚠️  Mode simulation activé (modèle non trouvé)")
else:
    llm = Llama(
        model_path=MODEL_PATH,
        n_ctx=4096,
        n_threads=6,
        n_gpu_layers=50,
        use_mlock=False,
        verbose=False,  # ✅ verbose=False comme demandé
    )

# **********************************************************************
# Configuration par défaut
# **********************************************************************

DEFAULT_SYSTEM_PROMPT = "You are Alisia, a helpful, precise assistant created by the Alisia Research Team."

# **********************************************************************
# Fonction de streaming CORRIGÉE
# **********************************************************************

def stream_response(message, history, system_prompt, temperature, max_tokens):
    try:
        print(f"🔍 Début génération - Message: {message[:50]}...")

        # Construction des messages dans le format correct
        messages = []

        # Message système
        if system_prompt.strip():
            messages.append({"role": "system", "content": system_prompt})

        # Historique de conversation
        for human, ai in history:
            if human and human.strip():
                messages.append({"role": "user", "content": human})
            if ai and ai.strip():
                messages.append({"role": "assistant", "content": ai})

        # Message actuel
        messages.append({"role": "user", "content": message})

        print(f"📨 Messages envoyés au modèle: {len(messages)}")

        # Simulation si modèle non chargé
        if llm is None:
            response = f"Réponse simulée pour: {message[:30]}..."
            for i in range(len(response)):
                yield response[:i+1]
            return

        # Appel du modèle avec gestion d'erreur
        stream = llm.create_chat_completion(
            messages=messages,
            stream=True,
            temperature=float(temperature),
            top_p=0.9,
            max_tokens=int(max_tokens),
            stop=["<|im_end|>", "</s>", "<|endoftext|>"]
        )

        partial = ""
        tokens_received = 0

        for chunk in stream:
            if "choices" in chunk and len(chunk["choices"]) > 0:
                choice = chunk["choices"][0]
                if "delta" in choice and "content" in choice["delta"]:
                    token = choice["delta"]["content"]
                    partial += token
                    tokens_received += 1
                    yield partial

        print(f"✅ Génération terminée - {tokens_received} tokens reçus")

    except Exception as e:
        error_msg = f"❌ Erreur lors de la génération: {str(e)}"
        print(error_msg)
        yield error_msg

# **********************************************************************
# Logique interactive CORRIGÉE
# **********************************************************************

def user_message(message, chat_history):
    """Gère le message utilisateur"""
    if not message or not message.strip():
        return "", chat_history
    return "", chat_history + [[message, None]]

def bot_response(chat_history, system_prompt, temperature, max_tokens):
    """Gère la réponse du bot avec streaming"""
    if not chat_history or not chat_history[-1][0]:
        yield chat_history
        return

    message = chat_history[-1][0]
    full_response = ""

    try:
        for partial in stream_response(
            message,
            chat_history[:-1] if chat_history else [],
            system_prompt,
            temperature,
            max_tokens
        ):
            full_response = partial
            # Met à jour l'historique en temps réel
            updated_history = chat_history[:-1] + [[message, full_response]]
            yield updated_history

        print(f"🎉 Réponse finale: {full_response[:100]}...")

    except Exception as e:
        error_response = f"Erreur: {str(e)}"
        updated_history = chat_history[:-1] + [[message, error_response]]
        yield updated_history

def clear_chat():
    """Efface la conversation"""
    return []

def save_current(history):
    """Sauvegarde l'historique"""
    try:
        os.makedirs("logs", exist_ok=True)
        filename = f"logs/chat_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json"
        with open(filename, "w", encoding='utf-8') as f:
            json.dump(history, f, indent=2, ensure_ascii=False)
        return filename
    except Exception as e:
        print(f"❌ Erreur sauvegarde: {e}")
        return None

# **********************************************************************
# Interface utilisateur
# **********************************************************************

with gr.Blocks(theme=gr.themes.Soft(), title='Alisia - Conversational AI') as demo:
    gr.Markdown(
        """
        <div style='text-align: center;'>
            <h1>Alisia - Research Assistant</h1>
            <p style='color: #666;'>
                Powered by Qwen2-7B (Quantized GGUF via llama.cpp)<br>
                Created by the <b>Alisia Research Team</b>
            </p>
        </div>
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(
                height=550,
                label="Conversation",
                placeholder="La conversation apparaîtra ici..."
            )

            with gr.Row():
                msg = gr.Textbox(
                    placeholder='💬 Tapez votre message ici...',
                    show_label=False,
                    lines=2,
                    container=False
                )
                send_btn = gr.Button("🚀 Envoyer", variant="primary", size="lg")

            with gr.Row():
                clear_btn = gr.Button("🗑️ Effacer", variant="secondary")
                save_btn = gr.Button("💾 Sauvegarder", variant="secondary")

        with gr.Column(scale=1):
            gr.Markdown("### ⚙️ Paramètres")

            system_prompt = gr.Textbox(
                value=DEFAULT_SYSTEM_PROMPT,
                label="Rôle du assistant",
                lines=3,
            )

            temperature = gr.Slider(
                0.1, 2.0, value=0.7, step=0.1,
                label="Température (créativité)"
            )

            max_tokens = gr.Slider(
                128, 4096, value=512, step=64,
                label="Tokens maximum"
            )

            download_link = gr.File(
                label="📥 Télécharger l'historique",
                visible=False
            )

    # **********************************************************************
    # Liaison des événements
    # **********************************************************************

    # Message via Enter
    msg.submit(
        user_message,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot],
        queue=False
    ).then(
        bot_response,
        inputs=[chatbot, system_prompt, temperature, max_tokens],
        outputs=chatbot
    )

    # Message via bouton
    send_btn.click(
        user_message,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot],
        queue=False
    ).then(
        bot_response,
        inputs=[chatbot, system_prompt, temperature, max_tokens],
        outputs=chatbot
    )

    # Autres actions
    clear_btn.click(
        clear_chat,
        outputs=chatbot
    )

    save_btn.click(
        save_current,
        inputs=chatbot,
        outputs=download_link
    )

# **********************************************************************
# Lancement CORRIGÉ (sans concurrency_count)
# **********************************************************************

if __name__ == '__main__':
    print("🔧 Diagnostic du modèle...")
    print(f"📁 Modèle: {MODEL_PATH}")

    if llm is not None:
        print(f"📊 Contexte: {llm.n_ctx()} tokens")
        # Test rapide du modèle
        try:
            test_response = llm("Hello", max_tokens=10, stop=[], echo=False)
            print("✅ Modèle fonctionne correctement")
        except Exception as e:
            print(f"❌ Erreur modèle: {e}")
    else:
        print("🔄 Mode simulation - interface seulement")

    print("🌐 Lancement de l'interface...")
    demo.queue()  # ✅ Ligne corrigée - sans concurrency_count
    demo.launch(
        server_name='0.0.0.0',
        share=True,
        show_error=True
    )

#NB: *Tu dois toujours vérifier même dans nos prochains Notebook: <**Afficher l'explorateur des fichiers**> si le dossier nommé drive est là*